<a href="https://colab.research.google.com/github/wongzw/IS4242_Group_4/blob/develop/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import packages

In [1]:
from joblib import dump, load
import os
from PIL import Image
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score, accuracy_score
from sklearn.svm import SVC

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import albumentations as A

transform = A.Compose([
    A.RandomBrightnessContrast(),
    A.Affine(scale=[0.8,1.2],translate_percent=0.05, shear=0.2, keep_ratio=True, p=0.5),
    A.Rotate(limit=10)
])

In [4]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 30.8 MB/s eta 0:00:00


In [5]:
# For capturing hand coordinates
import cv2
import mediapipe as mp

## Data Preprocessing

#### Read image and convert it to 28x28 matrix:

In [6]:
# folder_name = '/content/drive/MyDrive/IS4242/data' #might be different for different people
# path = folder_name
# img_list = []
# label_list = []
# txt = 'abcdefghiklmnopqrstuvwxy'

# for i in tqdm(range(len(os.listdir(path)))):
#   filename = os.listdir(path)[i]
#   if filename.endswith(('.jpg','.png')):
#       label_name = re.split(r'[-_]', filename)[0].lower()
#       if label_name not in txt:
#         continue
#       label_list.append(label_name)
#       img = Image.open(os.path.join(path, filename))
#       img = img.resize((28, 28), Image.ANTIALIAS)
#       img = img.convert('L')
#       img_array = np.array(img)
      
#       img_list.append(img_array)

# X = np.array(img_list)

# ##added 
# X = X/255

# y = np.array(label_list)
# y = np.array(list(map(lambda x:txt.find(x), y)))
# y = to_categorical(y)

In [7]:
output_dir = '/content/drive/MyDrive/IS4242/dump/'
X = load(output_dir+'X_224.pkl')
y = load(output_dir+'y_224.pkl')

In [8]:
len(X)

817

#### Train test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Augmentation only on train data

In [10]:
augmented_X = []
augmented_y = []
for i in range(len(X_train)):
  for j in (range(10)):
      transformed = transform(image=X_train[i])
      augmented_X.append(transformed['image'])
      augmented_y.append(y_train[i])

augmented_X = np.array(augmented_X)
augmented_y = np.array(augmented_y)
X_train = np.concatenate([X_train,augmented_X])
y_train = np.concatenate([y_train,augmented_y])

In [11]:
print(X_train.shape)
print(y_train.shape)

(7183, 224, 224)
(7183, 24)


In [12]:
print(y_train)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Using mediapipe

#### Generate dataset of hand points for train

In [13]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [14]:
data = []
labels = []

In [15]:
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_train))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_train[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_train[i]

                data.append(row)
                labels.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue


  0%|          | 13/7183 [00:00<04:04, 29.31it/s]

[0.5702299475669861, 0.7003850340843201, 0.46091753244400024, 0.6018336415290833, 0.3852296471595764, 0.5265490412712097, 0.32262295484542847, 0.47085893154144287, 0.30158817768096924, 0.4085491895675659, 0.5288745164871216, 0.3980141282081604, 0.4821246862411499, 0.291260302066803, 0.4099113643169403, 0.2953939139842987, 0.3609543442726135, 0.3214380741119385, 0.5508784651756287, 0.4108913242816925, 0.3696768283843994, 0.3695741891860962, 0.363323837518692, 0.448601633310318, 0.40807145833969116, 0.464846670627594, 0.5552369356155396, 0.4376516342163086, 0.375546932220459, 0.4221045970916748, 0.3878161311149597, 0.49417614936828613, 0.4347791075706482, 0.5040949583053589, 0.5495784878730774, 0.46931031346321106, 0.4166426658630371, 0.4679672122001648, 0.42391613125801086, 0.518278181552887, 0.4663311243057251, 0.5338477492332458] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


  0%|          | 24/7183 [00:00<04:24, 27.04it/s]

[0.44618257880210876, 0.7968925833702087, 0.3414516746997833, 0.6812290549278259, 0.30435711145401, 0.5003389716148376, 0.3140893578529358, 0.36659786105155945, 0.3157941699028015, 0.25381726026535034, 0.45740795135498047, 0.39108648896217346, 0.4473938047885895, 0.2733498215675354, 0.3924715518951416, 0.3526105284690857, 0.3555251657962799, 0.4469743072986603, 0.5481590032577515, 0.43036335706710815, 0.5281583070755005, 0.32074159383773804, 0.4578179717063904, 0.43469202518463135, 0.4192909002304077, 0.5372810363769531, 0.6377024054527283, 0.4878375828266144, 0.6066970229148865, 0.38676518201828003, 0.5277344584465027, 0.5071242451667786, 0.48841166496276855, 0.6016519069671631, 0.7347320318222046, 0.5641606450080872, 0.6992862224578857, 0.46973568201065063, 0.6206212639808655, 0.5442590713500977, 0.5745933055877686, 0.6150787472724915] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5548228621482849, 0.4673372805118561, 0.47795143723487854, 0.500206291675

  1%|          | 37/7183 [00:01<04:59, 23.85it/s]

...

[0.5653272271156311, 0.5064523220062256, 0.5158858299255371, 0.5207910537719727, 0.4617369472980499, 0.5547294020652771, 0.4306545555591583, 0.6005563139915466, 0.42050495743751526, 0.6411315202713013, 0.39854466915130615, 0.5280929803848267, 0.3306884467601776, 0.5946544408798218, 0.2850218117237091, 0.6280380487442017, 0.2471686601638794, 0.6549347043037415, 0.403281033039093, 0.5364893674850464, 0.39060524106025696, 0.6279560327529907, 0.4156208336353302, 0.6663267612457275, 0.43584755063056946, 0.6867556571960449, 0.4197577238082886, 0.5528881549835205, 0.4149218797683716, 0.635782778263092, 0.43710386753082275, 0.6705195307731628, 0.45791390538215637, 0.690823495388031, 0.44370973110198975, 0.5770242214202881, 0.44365665316581726, 0.6385247707366943, 0.45602068305015564, 0.6661479473114014, 0.47010889649391174, 0.6850878596305847] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.6008560061454773, 0.47101059556007385, 0.5516852140426636, 0.46917420625686

100%|█████████▉| 7180/7183 [05:47<00:00, 16.94it/s]

[0.5832306146621704, 0.45261916518211365, 0.5397135019302368, 0.4552246332168579, 0.49539482593536377, 0.4929042160511017, 0.4798513650894165, 0.5518550276756287, 0.48375722765922546, 0.6023728251457214, 0.4419454038143158, 0.46910321712493896, 0.3805476427078247, 0.5355685949325562, 0.3426836133003235, 0.5689758658409119, 0.31114906072616577, 0.5972999930381775, 0.44685208797454834, 0.4827183187007904, 0.43681877851486206, 0.5766763091087341, 0.4592565894126892, 0.6244124174118042, 0.4762648940086365, 0.6554614901542664, 0.463823139667511, 0.5011913776397705, 0.48196476697921753, 0.5834785103797913, 0.5190696716308594, 0.5858463048934937, 0.5451679825782776, 0.576425313949585, 0.4875549077987671, 0.5241711735725403, 0.5159078240394592, 0.5784797072410583, 0.5459681749343872, 0.5805479884147644, 0.5667844414710999, 0.5751457810401917] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5832306146621704, 0.45261916518211365, 0.5397135019302368, 0.455224633216857

100%|██████████| 7183/7183 [05:47<00:00, 20.67it/s]

[0.5470160245895386, 0.5090687274932861, 0.5071824789047241, 0.5260014533996582, 0.45492011308670044, 0.5577026605606079, 0.4246712625026703, 0.6028645038604736, 0.41304638981819153, 0.6439816355705261, 0.3947565257549286, 0.5175836682319641, 0.3187803328037262, 0.5771031975746155, 0.27158623933792114, 0.6065439581871033, 0.2315904051065445, 0.6301261186599731, 0.3959178924560547, 0.5230432748794556, 0.37200427055358887, 0.609275221824646, 0.3850063383579254, 0.6489843130111694, 0.39452674984931946, 0.6744517683982849, 0.4081595838069916, 0.5382407903671265, 0.40832197666168213, 0.6312914490699768, 0.4387575387954712, 0.6555348038673401, 0.46432220935821533, 0.6651104092597961, 0.4293968379497528, 0.5631586909294128, 0.436010479927063, 0.6311187744140625, 0.4543188810348511, 0.6569909453392029, 0.47032666206359863, 0.6740376949310303] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5832306146621704, 0.45261916518211365, 0.5397135019302368, 0.455224633216857

#### Generate hand points for test

In [16]:
test_data = []
test_label = []
count = 0
with mp_hands.Hands(static_image_mode =True, max_num_hands = 2, min_detection_confidence=0.5) as hands:
    for i in tqdm(range(len(X_test))):
        # Load the image and extract its features
            results = hands.process(cv2.cvtColor(cv2.flip(X_test[i],1), cv2.COLOR_BGR2RGB))
            try:
                # Extract Hand landmarks
                for hand_landmark in results.multi_hand_landmarks:
                    right_hand = hand_landmark.landmark
                right_hand_row = list(np.array([[landmark.x, landmark.y] for landmark in right_hand]).flatten())
                # Concate rows
                row = right_hand_row

                # Extract the label from the image filename (e.g. "A.jpg")
                label = y_test[i]

                test_data.append(row)
                test_label.append(label)
                print(row, label)
            except Exception as e:
                count += 1
                continue

  2%|▏         | 4/164 [00:00<00:12, 12.53it/s]

[0.5210281014442444, 0.641315221786499, 0.39546293020248413, 0.5578718781471252, 0.3307020962238312, 0.3800443112850189, 0.3052407503128052, 0.24902814626693726, 0.2702046036720276, 0.1634766012430191, 0.41695016622543335, 0.2820096015930176, 0.4198142886161804, 0.16589711606502533, 0.42496901750564575, 0.27493152022361755, 0.43036800622940063, 0.339459091424942, 0.5108478665351868, 0.2845029830932617, 0.5148842334747314, 0.17844367027282715, 0.4996938705444336, 0.3112812340259552, 0.49929216504096985, 0.34388941526412964, 0.5973529815673828, 0.31039708852767944, 0.6130174398422241, 0.20091238617897034, 0.5859422087669373, 0.32097747921943665, 0.5842393040657043, 0.3630259037017822, 0.6818413734436035, 0.3520607650279999, 0.6982240676879883, 0.2732140123844147, 0.6630502939224243, 0.35034728050231934, 0.6547291278839111, 0.3847343325614929] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5496582388877869, 0.6345279216766357, 0.47247281670570374, 0.625109791

  7%|▋         | 11/164 [00:00<00:08, 17.75it/s]

[0.5303622484207153, 0.7261391878128052, 0.3620838522911072, 0.6153806447982788, 0.2671659290790558, 0.40738582611083984, 0.22772908210754395, 0.24531611800193787, 0.17089855670928955, 0.12447883188724518, 0.40584099292755127, 0.294745534658432, 0.34948059916496277, 0.20392224192619324, 0.34238922595977783, 0.3307180404663086, 0.36016517877578735, 0.4387580454349518, 0.5203278064727783, 0.30517739057540894, 0.4636331796646118, 0.20133087038993835, 0.451718270778656, 0.3622964024543762, 0.46842825412750244, 0.47976988554000854, 0.6406235694885254, 0.32791975140571594, 0.569840133190155, 0.22479650378227234, 0.5453921556472778, 0.3856639564037323, 0.5597202777862549, 0.5043128728866577, 0.764388918876648, 0.36453112959861755, 0.6879178285598755, 0.2666337490081787, 0.6525718569755554, 0.3518322706222534, 0.6621406674385071, 0.42191678285598755] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.5895628929138184, 0.7734596133232117, 0.6003317832946777, 0.66376382

 13%|█▎        | 21/164 [00:01<00:06, 23.67it/s]

...

 90%|████████▉ | 147/164 [00:06<00:00, 24.99it/s]

[0.4025712013244629, 0.5869166851043701, 0.4375220537185669, 0.522254228591919, 0.5011380314826965, 0.4978063702583313, 0.5560323596000671, 0.5066971778869629, 0.5907210111618042, 0.523162841796875, 0.5285419225692749, 0.5317249298095703, 0.594545841217041, 0.5259853601455688, 0.6381144523620605, 0.5236724019050598, 0.6700278520584106, 0.5221999287605286, 0.5290641784667969, 0.5704972743988037, 0.5946837663650513, 0.5639264583587646, 0.6356530785560608, 0.556513249874115, 0.6702843308448792, 0.5516738891601562, 0.5267694592475891, 0.6048494577407837, 0.5903812050819397, 0.596366822719574, 0.632245659828186, 0.5849146842956543, 0.6661837100982666, 0.5781964659690857, 0.522779643535614, 0.6322373151779175, 0.5783458948135376, 0.6181328296661377, 0.6143463253974915, 0.6030421853065491, 0.643071711063385, 0.5925341248512268] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0.659817636013031, 0.7055771350860596, 0.7152523994445801, 0.5528687238693237, 0.67378699779

 94%|█████████▍| 154/164 [00:06<00:00, 26.83it/s]

[0.47691941261291504, 0.7220901846885681, 0.4109916090965271, 0.6569188833236694, 0.4090764820575714, 0.5418635010719299, 0.47629427909851074, 0.4535200595855713, 0.537406325340271, 0.3900882601737976, 0.37297624349594116, 0.46395376324653625, 0.38338690996170044, 0.41171514987945557, 0.375829815864563, 0.48565569519996643, 0.363042950630188, 0.5369953513145447, 0.4320586621761322, 0.46602949500083923, 0.44608330726623535, 0.4027238190174103, 0.4247105121612549, 0.49308809638023376, 0.3995841145515442, 0.5473983287811279, 0.4899352788925171, 0.4852662682533264, 0.506833553314209, 0.4309278130531311, 0.47783714532852173, 0.5107895135879517, 0.4487495720386505, 0.5549561381340027, 0.542538046836853, 0.515808641910553, 0.5471177697181702, 0.48352673649787903, 0.5181245803833008, 0.5462149977684021, 0.4944431781768799, 0.5775335431098938] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


100%|██████████| 164/164 [00:06<00:00, 24.34it/s]


In [17]:
labels = np.array(labels)
test_label = np.array(test_label)

In [18]:
x_train = np.array(data)
x_test = np.array(test_data)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
print(x_train.shape)
print(x_test.shape)

(2431, 42, 1)
(44, 42, 1)


#### SVM - linear


In [19]:
# linear model

model_linear = SVC(kernel='linear', probability=True)
model_linear.fit(x_train.reshape((x_train.shape[0],42)), np.argmax(labels, axis=1))

# predict
y_pred = model_linear.predict(x_test.reshape((x_test.shape[0],42)))

##### Evaluation

In [20]:
# confusion matrix and accuracy

# accuracy
print("accuracy:", accuracy_score(y_true=np.argmax(test_label, axis=1), y_pred=y_pred), "\n")

# cm
print(confusion_matrix(y_true=np.argmax(test_label, axis=1), y_pred=y_pred))

accuracy: 0.8181818181818182 

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
print(classification_report(np.argmax(test_label, axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      1.00      1.00         1
           2       1.00      0.50      0.67         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         1
          10       1.00      1.00      1.00         3
          11       0.50      1.00      0.67         1
          12       0.00      0.00      0.00         2
          13       0.67      1.00      0.80         2
          14       1.00      1.00      1.00         3
          16       0.50      0.67      0.57         3
          17       0.00      0.00      0.00         1
          18       0.67      0.67      0.67         3
          19       0.50    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# get auroc score

y_pred_scores = model_linear.predict_proba(x_test.reshape((x_test.shape[0],42)))
roc_auc_micro = roc_auc_score(test_label, y_pred_scores, multi_class='ovr', average='micro')
print("ROC AUC score (micro average)", roc_auc_micro)

ROC AUC score (micro average) 0.9919151994250809


#### SVM - rbf non linear 

In [23]:
# non-linear model
# using rbf kernel, C=1, default value of gamma
non_linear = SVC(kernel='rbf', probability=True)
non_linear.fit(x_train.reshape((x_train.shape[0],42)), np.argmax(labels, axis=1))

# predict
y_pred = non_linear.predict(x_test.reshape((x_test.shape[0],42)))

##### Evaluation

In [24]:
# confusion matrix and accuracy

# accuracy
print("accuracy:", accuracy_score(y_true=np.argmax(test_label, axis=1), y_pred=y_pred), "\n")

# cm
print(confusion_matrix(y_true=np.argmax(test_label, axis=1), y_pred=y_pred))

accuracy: 0.8409090909090909 

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [25]:
print(classification_report(np.argmax(test_label, axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         1
          10       1.00      0.67      0.80         3
          11       0.33      1.00      0.50         1
          12       0.00      0.00      0.00         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         3
          16       0.75      1.00      0.86         3
          17       0.33      1.00      0.50         1
          18       0.67      0.67      0.67         3
          19       1.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
# get auroc score

y_pred_scores = non_linear.predict_proba(x_test.reshape((x_test.shape[0],42)))
roc_auc_micro = roc_auc_score(test_label, y_pred_scores, multi_class='ovr', average='micro')
print("ROC AUC score (micro average)", roc_auc_micro)

ROC AUC score (micro average) 0.9934872439813152


#### SVM - GridSearchCV to get the best parameters

In [27]:
#add gridsearchcv

param_grid = {'C': [0.1, 1, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel':['rbf','poly','sigmoid','linear'],
              'degree':[1, 2, 3]}
grid = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid)
grid.fit(x_train.reshape((x_train.shape[0],42)), np.argmax(labels, axis=1))
# predict
y_pred = grid.predict(x_test.reshape((x_test.shape[0],42)))

In [28]:
print(grid.best_params_)

{'C': 100, 'degree': 2, 'gamma': 1, 'kernel': 'poly'}


In [29]:
# confusion matrix and accuracy

# accuracy
print("accuracy:", accuracy_score(y_true=np.argmax(test_label, axis=1), y_pred=y_pred), "\n")

# cm
print(confusion_matrix(y_true=np.argmax(test_label, axis=1), y_pred=y_pred))

accuracy: 0.8636363636363636 

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [30]:
print(classification_report(np.argmax(test_label, axis=1), y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         1
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         1
          12       1.00      0.50      0.67         2
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         3
          16       0.50      0.67      0.57         3
          17       0.00      0.00      0.00         1
          18       1.00      1.00      1.00         3
          19       0.00    

In [31]:
# get auroc score

y_pred_scores = grid.predict_proba(x_test.reshape((x_test.shape[0],42)))
roc_auc_micro = roc_auc_score(test_label, y_pred_scores, multi_class='ovr', average='micro')
print("ROC AUC score (micro average)", roc_auc_micro)

ROC AUC score (micro average) 0.9910842615882142


In [32]:
# s = dump(non_linear,'/content/drive/MyDrive/SVM/model.joblib')
s = dump(grid, '/content/drive/MyDrive/SVM/model.joblib')